<a href="https://colab.research.google.com/github/InSpira4/CS5617-DataScienceProject/blob/sugi/Descriptive_Sugi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Natera.csv');
df.head()

,Ticket Number,Date of Service,Carrier Code,Charges,Payments,Insurance Payment,Patient Payment,Writeoffs,FinancialClass,CPT,Date of Entry,Payment Post Date,NPI,Ordering Clinic,Denied Code,First Billed Date,Provider Profile,Insurance Bal,Patient Bal,Allowed Amount,Case Count
0,104452,1/2/2019,CFHP-E2,"$3,900.00",$0.00,$0.00,$0.00,$0.00,NC - NON-CONTRACTED,81507,1/7/2019,1/22/2019,1316428238,ALAMO WOMEN'S OBGYN,197,1/8/2019,NTINC,$0.00,"$3,900.00",$0.00,1
1,104453,1/2/2019,CFHP-E2,"$3,900.00",$0.00,$0.00,$0.00,$0.00,NC - NON-CONTRACTED,81507,1/7/2019,1/24/2019,1316428238,ALAMO WOMEN'S OBGYN,197,1/8/2019,NTINC,$0.00,"$3,900.00",$0.00,1
2,104454,1/2/2019,NCS,$795.00,$249.00,$0.00,$249.00,$546.00,SP - SELF PAY,99999,1/7/2019,2/10/2019,1699760116,COMPREHENSIVE WOMEN'S HEALTHCARE,-,-,NTINC,$0.00,$0.00,$0.00,1
3,104496,1/2/2019,AETLIF-E,"$3,900.00",$0.00,$0.00,$0.00,$0.00,CT - CONTRACTED,81420,1/7/2019,1/28/2019,1609216597,PEACHTREE WOMEN'S CLINIC - NORTHSIDE,-,1/8/2019,NTINC,"$3,900.00",$0.00,$749.00,1
4,104497,1/2/2019,AETLIF-E,"$3,900.00",$0.00,$0.00,$0.00,$0.00,CT - CONTRACTED,81420,1/7/2019,1/30/2019,1609216597,PEACHTREE WOMEN'S CLINIC - NORTHSIDE,-,1/8/2019,NTINC,"$3,900.00",$0.00,$749.00,1


In [0]:
cols = ['Payments', 'Charges', 'Insurance Payment', 'Patient Payment', 'Writeoffs', 'Insurance Bal', 'Patient Bal', 'Allowed Amount']
for col in cols:
  df[col] = pd.to_numeric(df[col].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(' ','')).apply(lambda x: x.replace(',','')).apply(lambda x: x.replace('(','')).apply(lambda x: x.replace(')','')))

In [0]:
cols = ['Date of Service', 'Date of Entry', 'Payment Post Date', 'First Billed Date']
for col in cols:
  df[col] = pd.to_datetime(df[col], errors='coerce')

In [0]:
cols = ['Ticket Number', 'Carrier Code', 'FinancialClass', 'CPT', 'NPI', 'Ordering Clinic', 'Denied Code', 'Provider Profile']
for col in cols:
  df[col] = df[col].astype('str')

In [0]:
def checkStatus(row):
  if(row['Denied Code'].strip() == '-'  and row['Payment Post Date'] != pd.NaT):
    return True
  elif(row['Denied Code'].strip() != '-'):
    return False
  else:
    return NaN
df['accepted'] = df.apply(lambda row: checkStatus(row), axis=1)

In [0]:
df.head()

,Ticket Number,Date of Service,Carrier Code,Charges,Payments,Insurance Payment,Patient Payment,Writeoffs,FinancialClass,CPT,Date of Entry,Payment Post Date,NPI,Ordering Clinic,Denied Code,First Billed Date,Provider Profile,Insurance Bal,Patient Bal,Allowed Amount,Case Count,accepted
0,104452,2019-01-02,CFHP-E2,3900.0,0.0,0.0,0.0,0.0,NC - NON-CONTRACTED,81507,2019-01-07,2019-01-22,1316428238,ALAMO WOMEN'S OBGYN,197,2019-01-08,NTINC,0.0,3900.0,0.0,1,False
1,104453,2019-01-02,CFHP-E2,3900.0,0.0,0.0,0.0,0.0,NC - NON-CONTRACTED,81507,2019-01-07,2019-01-24,1316428238,ALAMO WOMEN'S OBGYN,197,2019-01-08,NTINC,0.0,3900.0,0.0,1,False
2,104454,2019-01-02,NCS,795.0,249.0,0.0,249.0,546.0,SP - SELF PAY,99999,2019-01-07,2019-02-10,1699760116,COMPREHENSIVE WOMEN'S HEALTHCARE,-,NaT,NTINC,0.0,0.0,0.0,1,True
3,104496,2019-01-02,AETLIF-E,3900.0,0.0,0.0,0.0,0.0,CT - CONTRACTED,81420,2019-01-07,2019-01-28,1609216597,PEACHTREE WOMEN'S CLINIC - NORTHSIDE,-,2019-01-08,NTINC,3900.0,0.0,749.0,1,True
4,104497,2019-01-02,AETLIF-E,3900.0,0.0,0.0,0.0,0.0,CT - CONTRACTED,81420,2019-01-07,2019-01-30,1609216597,PEACHTREE WOMEN'S CLINIC - NORTHSIDE,-,2019-01-08,NTINC,3900.0,0.0,749.0,1,True


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10357 entries, 0 to 10356
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Ticket Number      10357 non-null  object        
 1   Date of Service    10357 non-null  datetime64[ns]
 2   Carrier Code       10357 non-null  object        
 3   Charges            10357 non-null  float64       
 4   Payments           10357 non-null  float64       
 5   Insurance Payment  10357 non-null  float64       
 6   Patient Payment    10357 non-null  float64       
 7   Writeoffs          10357 non-null  float64       
 8   FinancialClass     10357 non-null  object        
 9   CPT                10357 non-null  object        
 10  Date of Entry      10357 non-null  datetime64[ns]
 11  Payment Post Date  6798 non-null   datetime64[ns]
 12  NPI                10357 non-null  object        
 13  Ordering Clinic    10357 non-null  object        
 14  Denied

In [0]:
df.corr(method ='pearson') 

,Charges,Payments,Insurance Payment,Patient Payment,Writeoffs,Insurance Bal,Patient Bal,Allowed Amount,Case Count,accepted
Charges,1.000000,0.208528,0.198686,0.066846,0.492110,0.559216,0.384802,-0.071715,NaN,-0.014118
Payments,0.208528,1.000000,0.981083,0.176778,0.314464,-0.098623,-0.038002,0.331976,NaN,0.164254
Insurance Payment,0.198686,0.981083,1.000000,-0.017107,0.291712,-0.092588,-0.033433,0.336650,NaN,0.157006
Patient Payment,0.066846,0.176778,-0.017107,1.000000,0.141020,-0.038633,-0.026292,0.002992,NaN,0.050092
Writeoffs,0.492110,0.314464,0.291712,0.141020,1.000000,-0.192298,-0.073470,0.074842,NaN,0.046706
Insurance Bal,0.559216,-0.098623,-0.092588,-0.038633,-0.192298,1.000000,-0.159482,-0.154776,NaN,0.093431
Patient Bal,0.384802,-0.038002,-0.033433,-0.026292,-0.073470,-0.159482,1.000000,-0.057735,NaN,-0.239589
Allowed Amount,-0.071715,0.331976,0.336650,0.002992,0.074842,-0.154776,-0.057735,1.000000,NaN,0.161597
Case Count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
accepted,-0.014118,0.164254,0.157006,0.050092,0.046706,0.093431,-0.239589,0.161597,NaN,1.000000


In [0]:
df.corr(method ='kendall') 

,Charges,Payments,Insurance Payment,Patient Payment,Writeoffs,Insurance Bal,Patient Bal,Allowed Amount,Case Count,accepted
Charges,1.000000,0.059643,0.038721,0.060145,0.184642,0.220420,0.150847,-0.025493,NaN,-0.019514
Payments,0.059643,1.000000,0.940882,0.282584,0.464715,-0.298644,-0.089334,0.509951,NaN,0.290958
Insurance Payment,0.038721,0.940882,1.000000,-0.039976,0.429472,-0.283826,-0.075130,0.525332,NaN,0.283381
Patient Payment,0.060145,0.282584,-0.039976,1.000000,0.160747,-0.087313,-0.055414,0.028119,NaN,0.065016
Writeoffs,0.184642,0.464715,0.429472,0.160747,1.000000,-0.472288,0.041144,0.384831,NaN,0.086385
Insurance Bal,0.220420,-0.298644,-0.283826,-0.087313,-0.472288,1.000000,-0.503083,-0.347402,NaN,0.179521
Patient Bal,0.150847,-0.089334,-0.075130,-0.055414,0.041144,-0.503083,1.000000,0.089637,NaN,-0.275555
Allowed Amount,-0.025493,0.509951,0.525332,0.028119,0.384831,-0.347402,0.089637,1.000000,NaN,0.066801
Case Count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
accepted,-0.019514,0.290958,0.283381,0.065016,0.086385,0.179521,-0.275555,0.066801,NaN,1.000000


In [0]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Set default Seaborn style
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
yes = []
no = []
labels = df['FinancialClass'].unique()
count = df
for label in labels:
    print(label, df[df['FinancialClass'] == label].accepted.sum())
    yes.append(df[df['FinancialClass'] == label].accepted.sum())
    no.append(df[df['FinancialClass'] == label].accepted.count() - df[df['FinancialClass'] == label].accepted.sum())
print(yes)
print(no)

NC - NON-CONTRACTED 1547
SP - SELF PAY 212
CT - CONTRACTED 3212
MD - MEDICAID 704
BC - BLUE CROSS/BLUE SHIELD 484
MM - MANAGED MEDICAID 20
HM - HMO 66
IP - IPA 37
MC - MEDICARE 16
[1547, 212, 3212, 704, 484, 20, 66, 37, 16]
[949, 0, 1881, 892, 204, 45, 88, 0, 0]


In [0]:
raw_data = {'labels': labels,
        'true': yes,
        'no': no}
df1 = pd.DataFrame(raw_data)
df1

,labels,true,no
0,NC - NON-CONTRACTED,1547,949
1,SP - SELF PAY,212,0
2,CT - CONTRACTED,3212,1881
3,MD - MEDICAID,704,892
4,BC - BLUE CROSS/BLUE SHIELD,484,204
5,MM - MANAGED MEDICAID,20,45
6,HM - HMO,66,88
7,IP - IPA,37,0
8,MC - MEDICARE,16,0


In [0]:
df[df['accepted'] == True].count()

Ticket Number        6298
Date of Service      6298
Carrier Code         6298
Charges              6298
Payments             6298
Insurance Payment    6298
Patient Payment      6298
Writeoffs            6298
FinancialClass       6298
CPT                  6298
Date of Entry        6298
Payment Post Date    2741
NPI                  6298
Ordering Clinic      6298
Denied Code          6298
First Billed Date    6040
Provider Profile     6298
Insurance Bal        6298
Patient Bal          6298
Allowed Amount       6298
Case Count           6298
accepted             6298
dtype: int64

In [0]:
df.count()

Ticket Number        10357
Date of Service      10357
Carrier Code         10357
Charges              10357
Payments             10357
Insurance Payment    10357
Patient Payment      10357
Writeoffs            10357
FinancialClass       10357
CPT                  10357
Date of Entry        10357
Payment Post Date     6798
NPI                  10357
Ordering Clinic      10357
Denied Code          10357
First Billed Date    10099
Provider Profile     10357
Insurance Bal        10357
Patient Bal          10357
Allowed Amount       10357
Case Count           10357
accepted             10357
dtype: int64

In [0]:
4059 + 6298

10357